In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Swapn\\Documents\\ProjectDrive\\IndustryProject\\TextSummarizer-using-HuggingFace-Transformer\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Swapn\\Documents\\ProjectDrive\\IndustryProject\\TextSummarizer-using-HuggingFace-Transformer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        param_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        
        create_directories([self.config.artifacts_root])
        
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config =  DataIngestionConfig(
            root_dir = self.config.artifacts_root,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [16]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [17]:
class DataIngetion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! With following info: \n{headers}")
        else:
            logger.info(f"File already exists : {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        """
        zip file is extracted to the unzip_dir
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngetion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-24 22:00:56,966: INFO: common: Successfully read yaml file: config\config.yaml]
[2023-07-24 22:00:56,969: INFO: common: Successfully read yaml file: params.yaml]
[2023-07-24 22:00:56,973: INFO: common: Created Directory: artifacts]
[2023-07-24 22:00:56,976: INFO: common: Created Directory: artifacts/data_ingestion]
[2023-07-24 22:01:09,457: INFO: 1938602191: artifacts/data_ingestion/data.zip download! With following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 28C0:69BED:67AD:8D5D:64BEA740
Accept-Ranges: bytes
Date: Mon, 24 Jul 2023 16:30:58 GMT
Via: 1.1 varnish
X-Served-By: cache-del21721-DEL
X-Cache: MISS
X-Cache-Hit